## Metaclasses
- Metaclasses are classes whose instance are class objects.
- Allows for dynamic creation of classes e.g dynamically adding/removing methods and creating logic at runtime.
- Gives control of how child classes are being created, invisible to the person implementing the child class.

In [1]:
import itertools

def exec_(command):
    """Print command and result of execution"""
    print(command + ": " + str(eval(command)))
    
def members(obj):
    """Return public members as strings"""
    return [member for member in dir(obj) if not member.startswith('__')]

### 1. Creating classes via type

In [2]:
def init(self, x):
    self.x = x
    
def inc(self):
    self.x += 1
    return self.x

A = type('A', (object, ), {'__init__': init, 'inc': inc})
a = A(0)

exec_('a.__class__.__name__')

exec_('a.inc()')
exec_('a.inc()')

exec_('a.__dict__')
exec_('members(a)')

a.__class__.__name__: A
a.inc(): 1
a.inc(): 2
a.__dict__: {'x': 2}
members(a): ['inc', 'x']


#### 1.1 Usecase: Class factory
 - Dynamically creates classes and configures what methods they should have.

In [3]:
def method1(self):
    print('method1')

def method2(self):
    print('method2')

class ClassFactory(object):
    
    def create(self, with_method1=False, with_method2=False):
        name = 'C'
        dict_ = {}
        if with_method1:
            dict_['method1'] = method1
            name += 'M1'
        if with_method2:
            dict_['method2'] = method2
            name += 'M2'
        res = type(name, (object, ), dict_)
        return res
        
        
factory = ClassFactory()
confs = itertools.product([False, True], [False, True])
classes = [factory.create(*conf) for conf in confs]

for C in classes:
    exec_('C.__name__')
    exec_('members(C)')
    print('')

C.__name__: C
members(C): []

C.__name__: CM2
members(C): ['method2']

C.__name__: CM1
members(C): ['method1']

C.__name__: CM1M2
members(C): ['method1', 'method2']



### 2. Metaclasses
- Overwrite the behaviour of type.
- Metaclass \_\_init\_\_ has same signature as type.
- When the code that contains the class definition is being executed it is taken apart and sent as parameters to the metaclass constructor.
    - name and bases extracted from class signature
    - dict_ contains the members defined in the class body
    

In [4]:
class MetaClass(type):
    
    def __init__(self, name, bases, dict_):
        """
        This is being called when the cell below is being executed.
        """
        super().__init__(name, bases, dict_)
        print('Class {} is being created'.format(name))
        print('These are the class parents: {}'.format(bases))
        print('These are the attributes from the class body {}'.format(dict_))
        self.class_variable_defined_in_metaclass = 10
        self.func_defined_in_metaclass = lambda self, x: self.class_variable_defined_in_metaclass + x

In [5]:
"""
The metaclass is being called when this code is passed to the interpreter aka when this cell is run.
! Not when A is being instantiated !
The result is that the object A (which represents a class) is being created and made available.
"""

class A(object, metaclass=MetaClass):
    """
    Some doc
    """
    def __init__(self):
        print('An object of type A is being instantiated')
        
    def print_members(self):
        print(members(self))

Class A is being created
These are the class parents: (<class 'object'>,)
These are the attributes from the class body {'__module__': '__main__', '__qualname__': 'A', '__doc__': '\n    Some doc\n    ', '__init__': <function A.__init__ at 0x7fabf0420d90>, 'print_members': <function A.print_members at 0x7fabf0420950>}


In [6]:
A.class_variable_defined_in_metaclass

10

In [7]:
a = A()

An object of type A is being instantiated


In [8]:
a.print_members()

['class_variable_defined_in_metaclass', 'func_defined_in_metaclass', 'print_members']


#### 2.1 Usecase: Wrap methods from child classes
- How to wrap a method in any child class, preferably in a transparent manner.
    - Option 1: decorate the method in child class.
    - Option 2: wrap child method in parent init: gets messed up in serialization/deserialization.
    - Option 3: have a func in parent class which calls a func_ overriden in child class.
    - Option 4 not valid: override parent func in child class and call super().func - doesn't work if you want things to happend before and after execution.
    - Option 5: meta classes!
- Control the creation of a child class by wrapping at class instantiation.

In [9]:
def wrapper(func):
    def wrapped(self, *args, **kwargs):
        print('wrapper: ' + func.__name__ + ' will be called')
        res = func(self, *args, **kwargs)
        print('wrapper: ' + func.__name__ + ' has been called')
        return res
    return wrapped

class MetaClass(type):
    
    def __init__(self, name, bases, dict_):
        super().__init__(name, bases, dict_)
        
        if 'func' in dict_:
            print('wrapping func of {}'.format(name))
            self.func = wrapper(dict_['func'])
        

In [10]:
class A(object, metaclass=MetaClass):
    def __init__(self, x):
        self.x = x
        
    def func(self):
        print(self.x)

wrapping func of A


In [11]:
A(99).func()

wrapper: func will be called
99
wrapper: func has been called


In [12]:
# But also in any subsequent child class

class B(A):
    def __init__(self, x):
        self.x = x
        
    def func(self):
        print(self.x)

B(99).func()

wrapping func of B
wrapper: func will be called
99
wrapper: func has been called
